# Формирование БД с информации о продажах вторичного жилья

In [41]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
import pandas as pd
import numpy as np

In [42]:
%%HTML
<style>.dataframe th{background: rgb(63,87,124);background: linear-gradient(180deg, rgba(63,87,124,1) 0%, rgba(101,124,161,1) 100%, rgba(0,212,255,1) 100%);;
padding: 10px;font-family: monospace;font-size: 110%;color: white;border:1px dashed white;text-align:left !important;
-moz-border-radius: 3x;-webkit-border-radius: 3px;}.dataframe thead{border:none; !important;}</style>

### Сбор ссылок на объявления, размещенных на сайте "Яндекс-недвижимость"

In [2]:
def get_links(url, i):
    browser.get(url+f'?page={i}')
    page = BeautifulSoup(browser.page_source, 'html.parser')
    
    link_tmp=[]
    
    links = page.find_all('div','OffersSerpItem__generalInfo')
    
    for link in links:
        link_tmp.append('https://realty.yandex.ru'+link.find('a').get('href'))        

    return link_tmp

In [3]:
def get_links_df(url):
    link_list = []
    
    for n in range(1,depth+1):
        link = get_links(url, n)      
        
        link_list += link
        
    df = pd.DataFrame(
    {  
        'link': link_list
    })
    
    return df   

In [4]:
url = 'https://realty.yandex.ru/perm/kupit/kvartira/vtorichniy-rynok/'

In [5]:
browser = Chrome('chromedriver.exe')

In [6]:
depth = 1

In [7]:
df = get_links_df(url)

In [8]:
browser.close()

In [9]:
df.to_csv('links_yandex.csv',sep=',', encoding='utf-8', header=True, index=False)

### Сбор информации по каждому отдельному объявлению

In [10]:
df = pd.read_csv('links_yandex.csv',sep=',')

In [11]:
browser = Chrome('chromedriver.exe')

In [12]:
def get_content(url):
    browser.get(url)
    page = BeautifulSoup(browser.page_source, 'html.parser')
    
    name_tmp = []
    place_tmp = []
    address_tmp = []
    price_tmp = []
    price_per_m_tmp = []
    tech_descr_tmp=[]
    detail_descr_tmp = []
    build_tmp = []
    
    try:
        place = page.find_all('span','Breadcrumbs__item-name')[3]
        place_tmp = place.text
    except: place_tmp=""
    
    try:
        name_tmp=page.find('h1', 'OfferHeader__title').text
    except: name_tmp = ""
        
    try:
        address_tmp=page.find('div', 'OfferHeader__address').text
    except: address_tmp = ""
        
    try:
        price_tmp=page.find('span', 'price').text.replace("\xa0","")
    except: price_tmp = ""
     
    try:
        price_per_m_tmp=page.find('div', 'OfferBaseInfo__text-info').text.replace("\xa0","")
    except: price_per_m_tmp = ""
    
    try:
        date_tmp=page.find('div', 'OfferPublishedDate OfferBaseMetaInfo__item').text.replace("\xa0","")
    except: date_tmp = ""
        
    try:
        ulTag = page.find("div", "OfferTechDescription OfferDescription__section").find("ul", {"class":"ColumnsList OfferTechDescription__list"})
        for liTag in ulTag.find_all("li", {"class":"ColumnsList__list-item"}):
            tech_descr_tmp.append(liTag.find("div", {"class":"ColumnsList__item"}).text.replace("\xa0",""))
    except: tech_descr_tmp=""
        
    try:
        ulTag = page.find("div", "OfferDetailsDescription OfferDescription__section").find("ul", {"class":"ColumnsList"})
        for liTag in ulTag.find_all("li", {"class":"ColumnsList__list-item"}):
            detail_descr_tmp.append(liTag.find("span", {"class":"OfferDetailsDescription__feature"}).text.replace("\xa0",""))
    except: detail_descr_tmp=""
        
    try:
        for tag in page.find("ul","ColumnsList BuildingInfo__features-list").find_all("span","BuildingInfoFeature__by-yandex-data"):
            build_tmp.append(tag.text.replace(" (по данным Яндекса)", "").replace("\xa0",""))
    except: build_tmp = ""
        
    return name_tmp, place_tmp, address_tmp, price_tmp, price_per_m_tmp, date_tmp, tech_descr_tmp, detail_descr_tmp, build_tmp

In [13]:
name_list = []
place_list = []
address_list = []
price_list = []
date_list = []
price_per_m_list = []
tech_descr_list = []
detail_descr_list = []
build_list = []
    
for i in df.link:
    name_tmp, place_tmp, address_tmp, price_tmp, price_per_m_tmp, date_tmp, tech_descr_tmp, detail_descr_tmp, build_tmp = get_content(i) 
        
    name_list.append(name_tmp)
    place_list.append(place_tmp)
    address_list.append(address_tmp)
    price_list.append(price_tmp)
    date_list.append(date_tmp)
    price_per_m_list.append(price_per_m_tmp)
    tech_descr_list.append(tech_descr_tmp)
    detail_descr_list.append(detail_descr_tmp)
    build_list.append(build_tmp)
        
df['name'] = name_list
df['place'] = place_list
df['address'] = address_list
df['price'] = price_list
df['date'] = date_list
df['price_per_m'] = price_per_m_list
df['tech'] = tech_descr_list
df['detail'] = detail_descr_list
df['building'] = build_list

In [14]:
browser.close()

In [15]:
df.to_csv('yandex.csv',sep=',', encoding='utf-8', header=True, index=False)

In [16]:
df

,link,name,place,address,price,date,price_per_m,tech,detail,building
0,https://realty.yandex.ru/offer/244102598290668...,"Купить 3-комнатную квартиру 79,6 м², 18/22 эта...",Ленинский район,"Пермь, улица Луначарского, 99",7300000 ₽,26февраля (обновлено 6мая),91709 ₽ зам²,"[79,6м² — общая площадь, 44,9м² — жилая, 13,9м...","[Санузел совмещённый, Две лоджии, Вид из окон ...","[Застройщик Талан, Дом 2020г., Монолитное здан..."
1,https://realty.yandex.ru/offer/403763061383238...,"Купить квартиру-студию 30 м², 9/25 этаж в ЖК «...",Орджоникидзевский район,"Пермь, улица Карбышева, 47Б",2400000 ₽,23марта (обновлено 4часаназад),80000 ₽ зам²,"[30м² — общая площадь, Студия, 9 этаж из 25]",,"[Застройщик Квартал, Монолитное здание, от 18 ..."
2,https://realty.yandex.ru/offer/522526019514858...,"Купить 1-комнатную квартиру 50,3 м², 2/24 этаж...",Ленинский район,"Пермь, улица Луначарского, 99",4500000 ₽,23марта (обновлено 4часаназад),89410 ₽ зам²,"[50,3м² — общая площадь, 10м² — кухня, 1 комна...",[Санузел раздельный],"[Застройщик Талан, Дом 2021г., Монолитное здан..."
3,https://realty.yandex.ru/offer/716916468062890...,"Купить 3-комнатную квартиру 58 м², 3/9 этаж",Мотовилихинский район,"Пермь, улица Звонарёва, 1",3920000 ₽,30марта (обновлено 6часовназад),67586 ₽ зам²,"[58м² — общая площадь, 39м² — жилая, 10м² — ку...",,"[Индивидуальный проект, Дом 1984г., Панельное ..."
4,https://realty.yandex.ru/offer/166213672882217...,"Купить 1-комнатную квартиру 39,6 м², 13/14 этаж",Индустриальный район,"Пермь, улица Космонавта Леонова, 56А",2750000 ₽,7мая (обновлено 15мая),69444 ₽ зам²,"[39,6м² — общая площадь, 1 комната, 13 этаж из...",,"[Индивидуальный проект, Дом 1995г., Монолитное..."
5,https://realty.yandex.ru/offer/505414989014770...,"Купить 1-комнатную квартиру 39 м², 19/25 этаж ...",Свердловский район,"Пермь, улица Солдатова, 29В",3290000 ₽,5часовназад,84359 ₽ зам²,"[39м² — общая площадь, 24м² — жилая, 6м² — кух...",,"[Застройщик ГК «ПМД», Дом 2018г., Монолитное з..."
6,https://realty.yandex.ru/offer/719703730336027...,"Купить 2-комнатную квартиру 61,7 м², 10/10 эта...",Дзержинский район,"Пермь, Вильвенская улица, 2",6300000 ₽,6часовназад,102107 ₽ зам²,"[61,7м² — общая площадь, 31,2м² — жилая, 16,3м...",,"[Застройщик Ясно девелопмент, Дом 2015г., Моно..."
7,https://realty.yandex.ru/offer/481714173977200...,"Купить 1-комнатную квартиру 30 м², 5/5 этаж",Мотовилихинский район,"Пермь, улица Добролюбова, 22",2390000 ₽,7часовназад,79667 ₽ зам²,"[30м² — общая площадь, 16,6м² — жилая, 6,2м² —...",,"[Индивидуальный проект, Дом 1969г., Кирпичное ..."
8,https://realty.yandex.ru/offer/799309322507314...,"Купить 1-комнатную квартиру 37,9 м², 19/25 эта...",Мотовилихинский район,"Пермь, улица Тургенева, 33Д",3700000 ₽,7часовназад,97625 ₽ зам²,"[37,9м² — общая площадь, 15м² — жилая, 12,7м² ...",,"[Застройщик ГК «ПМД», Дом 2020г., Монолитное з..."
9,https://realty.yandex.ru/offer/253927839475055...,"Купить 1-комнатную квартиру 31,3 м², 4/5 этаж",Свердловский район,"Пермь, улица Механошина, 14",2400000 ₽,7часовназад,76677 ₽ зам²,"[31,3м² — общая площадь, 19м² — жилая, 5,9м² —...",,"[Индивидуальный проект, Дом 1965г., Панельное ..."


In [17]:
new_columns = ['total_area', 'living_area', 'kitchen_area', 'rooms', 'floor', 'total_floors', 'bathroom', 'developer', 'year', 'build', 'ceiling', 'elevator', 'parking']

In [18]:
for col in new_columns:
    df[col] = ""*len(df)

In [19]:
for i in range(0, len(df)):
    
    t = df.building[i]
    
    for j in range(0, len(t)):
        if t[j].find('Застройщик')!=-1 or t[j].find('проект')!=-1:
            df.developer[i] = t[j]
        elif t[j].find('здание')!=-1:
            df.build[i] = t[j].replace(' здание','')
        elif t[j].find('Дом')!=-1:
            df.year[i] = int(t[j].replace('Дом','').replace('г.',''))
        elif t[j].find('потолки')!=-1:
            df.ceiling[i] = t[j].replace(' потолки','')
        elif t[j].find('лифт')!=-1 or t[j].find('Лифт')!=-1:
            df.elevator[i] = t[j].replace('Лифта нет','No').replace('Лифт','Yes')
        elif t[j].find('парковка')!=-1:
            df.parking[i] = t[j].replace(' парковка','')

In [20]:
for i in range(0, len(df)):
    t = df.tech[i]
    for item in t:
        if item.find('общая площадь')!=-1:
            df.total_area[i] = item.replace('— общая площадь','')
        elif item.find('комнат')!=-1:
            df.rooms[i] = item.replace('комнаты', '').replace('комната','')
        elif item.find('кухня')!=-1:
            df.kitchen_area[i] = item.replace('— кухня','')
        elif item.find('жилая')!=-1:
            df.living_area[i] = item.replace('— жилая','')
        elif item.find('этаж')!=-1:
            df.floor[i] = item.split()[0]
            df.total_floors[i] = item.split()[len(item.split())-1]

In [21]:
for i in range(0, len(df)):
    t = df.detail[i]
    for item in t:
        if item.find('Санузел')!=-1:
            df.bathroom[i] = item.replace('Санузел','')

In [22]:
df = df.drop(['tech','detail', 'building'],axis=1)

In [23]:
df.to_csv('yandex.csv',sep=',', encoding='utf-8', header=True, index=False)

In [24]:
df

,link,name,place,address,price,date,price_per_m,total_area,living_area,kitchen_area,rooms,floor,total_floors,bathroom,developer,year,build,ceiling,elevator,parking
0,https://realty.yandex.ru/offer/244102598290668...,"Купить 3-комнатную квартиру 79,6 м², 18/22 эта...",Ленинский район,"Пермь, улица Луначарского, 99",7300000 ₽,26февраля (обновлено 6мая),91709 ₽ зам²,"79,6м²","44,9м²","13,9м²",3,18,22,совмещённый,Застройщик Талан,2020,Монолитное,,Yes,Открытая
1,https://realty.yandex.ru/offer/403763061383238...,"Купить квартиру-студию 30 м², 9/25 этаж в ЖК «...",Орджоникидзевский район,"Пермь, улица Карбышева, 47Б",2400000 ₽,23марта (обновлено 4часаназад),80000 ₽ зам²,30м²,,,,9,25,,Застройщик Квартал,,Монолитное,,No,Открытая
2,https://realty.yandex.ru/offer/522526019514858...,"Купить 1-комнатную квартиру 50,3 м², 2/24 этаж...",Ленинский район,"Пермь, улица Луначарского, 99",4500000 ₽,23марта (обновлено 4часаназад),89410 ₽ зам²,"50,3м²",,10м²,1,2,24,раздельный,Застройщик Талан,2021,Монолитное,,No,Открытая
3,https://realty.yandex.ru/offer/716916468062890...,"Купить 3-комнатную квартиру 58 м², 3/9 этаж",Мотовилихинский район,"Пермь, улица Звонарёва, 1",3920000 ₽,30марта (обновлено 6часовназад),67586 ₽ зам²,58м²,39м²,10м²,3,3,9,,Индивидуальный проект,1984,Панельное,"2,6 м",Yes,Открытая
4,https://realty.yandex.ru/offer/166213672882217...,"Купить 1-комнатную квартиру 39,6 м², 13/14 этаж",Индустриальный район,"Пермь, улица Космонавта Леонова, 56А",2750000 ₽,7мая (обновлено 15мая),69444 ₽ зам²,"39,6м²",,,1,13,14,,Индивидуальный проект,1995,Монолитное,"2,5 м",Yes,Открытая
5,https://realty.yandex.ru/offer/505414989014770...,"Купить 1-комнатную квартиру 39 м², 19/25 этаж ...",Свердловский район,"Пермь, улица Солдатова, 29В",3290000 ₽,5часовназад,84359 ₽ зам²,39м²,24м²,6м²,1,19,25,,Застройщик ГК «ПМД»,2018,Монолитное,"2,7 м",Yes,Закрытая
6,https://realty.yandex.ru/offer/719703730336027...,"Купить 2-комнатную квартиру 61,7 м², 10/10 эта...",Дзержинский район,"Пермь, Вильвенская улица, 2",6300000 ₽,6часовназад,102107 ₽ зам²,"61,7м²","31,2м²","16,3м²",2,10,10,,Застройщик Ясно девелопмент,2015,Монолитное,"2,75 м",Yes,Открытая
7,https://realty.yandex.ru/offer/481714173977200...,"Купить 1-комнатную квартиру 30 м², 5/5 этаж",Мотовилихинский район,"Пермь, улица Добролюбова, 22",2390000 ₽,7часовназад,79667 ₽ зам²,30м²,"16,6м²","6,2м²",1,5,5,,Индивидуальный проект,1969,Кирпичное,"2,5 м",No,Открытая
8,https://realty.yandex.ru/offer/799309322507314...,"Купить 1-комнатную квартиру 37,9 м², 19/25 эта...",Мотовилихинский район,"Пермь, улица Тургенева, 33Д",3700000 ₽,7часовназад,97625 ₽ зам²,"37,9м²",15м²,"12,7м²",1,19,25,,Застройщик ГК «ПМД»,2020,Монолитное,"2,7 м",Yes,Закрытая
9,https://realty.yandex.ru/offer/253927839475055...,"Купить 1-комнатную квартиру 31,3 м², 4/5 этаж",Свердловский район,"Пермь, улица Механошина, 14",2400000 ₽,7часовназад,76677 ₽ зам²,"31,3м²",19м²,"5,9м²",1,4,5,,Индивидуальный проект,1965,Панельное,"2,5 м",No,Открытая


In [43]:
import folium
from tqdm import tqdm_notebook as tqdmn

In [27]:
from selenium import webdriver

In [28]:
df['Url'] = ['https://www.google.com/maps/search/' + i for i in df.address ]

In [30]:
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
# для ускорения (без загрузки картинок)
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':1}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome("chromedriver.exe", options=option)
browser = Chrome('chromedriver.exe')

for url in tqdmn(df.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

driver.close()

import csv

with open('Url_With_Coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(Url_With_Coordinates)

<ipython-input-30-f5368c7c79c6>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdmn(df.Url, leave=False):


  0%|          | 0/23 [00:00<?, ?it/s]

In [31]:
import csv

with open('Url_With_Coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        Url_With_Coordinates = i
        break

In [33]:
df['Url_With_Coordinates'] = Url_With_Coordinates

In [34]:
# устранение ошибки при попытке извлечь координаты из ссылки
# причина: наличие ссылок с отличающимся содержанием
df[~df.Url_With_Coordinates.str.contains('&zoom=')]

,link,name,place,address,price,date,price_per_m,total_area,living_area,kitchen_area,...,total_floors,bathroom,developer,year,build,ceiling,elevator,parking,Url,Url_With_Coordinates


In [35]:
# небольшое количество, поэтому удалим
df = df[df.Url_With_Coordinates.str.contains('&zoom=')].copy()

In [36]:
new_columns = ['lat', 'long']
for col in new_columns:
    df[col] = ""*len(df)

In [37]:
df['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in df['Url_With_Coordinates'] ]
df['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in df['Url_With_Coordinates'] ]

In [38]:
df = df.drop(['Url_With_Coordinates', 'Url', 'name'], axis=1)

In [39]:
df.to_csv('yandex_with_geo.csv',sep=',', encoding='utf-8', header=True, index=False)

In [45]:
df=pd.read_csv('yandex_with_geo.csv',sep=',', encoding='utf-8')

In [46]:
lat_list = df.lat
long_list = df.long
price_list = df.price

map = folium.Map(location=[lat_list[0],long_list[0]], zoom_start = 5)

for lat, long, price in zip(lat_list, long_list, price_list):
    folium.Marker(location=[lat, long], popup=str(price), icon=folium.Icon(color = 'gray')).add_to(map)

map.save("map_flat.html")